In [2]:
import numpy as np

m is number of features  
n is number of training examples  
X is of dimension (m,n) and Y is of dimension (m,1)

In [172]:
class lin_reg:
        
    def train(self, X, Y, alpha = 0.1):
        # Extract number of training examples and features from X
        (m, n) = X.shape
        # Add a column of ones to X for the bias
        X = np.append(np.ones((m,1)), X, axis = 1)
        Y = np.reshape(Y, (m,1))
        # Initialize the weights to zero
        weights = np.zeros((n+1 ,1))
        # Initialize the cost
        i = 0
        self.cost = []
        self.cost.append(1/m * np.matmul(np.transpose(Y - np.matmul(X, weights)), Y - np.matmul(X, weights)))
        while True:
            weights = weights + alpha / m * np.matmul(np.transpose(X), Y - np.matmul(X, weights))
            self.cost.append(1/m * np.matmul(np.transpose(Y - np.matmul(X, weights)), Y - np.matmul(X, weights)))
            if self.cost[i + 1] < self.cost[i]:
                i += 1
            else:
                break
            if i > 1000000:
                print('Gradient descent did not converge fast enough')
                break
        

In [38]:
    def predict(self, X):
        # Extract number of training examples and features from X
        (m, n) = X.shape
        # Add a column of ones to X for the bias
        X = np.append(np.ones((m,1)), X, axis = 1)

In [99]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
data = load_boston()

In [182]:
model = lin_reg()
model.train(data['data'], data['target'], alpha=0.000001)
model.cost

Gradient descent did not converge fast enough


[array([[592.146917]]),
 array([[349.66048201]]),
 array([[233.95288997]]),
 array([[178.13637393]]),
 array([[150.64094942]]),
 array([[136.56476656]]),
 array([[128.87251857]]),
 array([[124.24157953]]),
 array([[121.10160955]]),
 array([[118.70935353]]),
 array([[116.71199328]]),
 array([[114.94106971]]),
 array([[113.31522125]]),
 array([[111.79427765]]),
 array([[110.35759086]]),
 array([[108.99380459]]),
 array([[107.69602252]]),
 array([[106.45952444]]),
 array([[105.28068511]]),
 array([[104.15646093]]),
 array([[103.08414456]]),
 array([[102.06124634]]),
 array([[101.08543556]]),
 array([[100.15451013]]),
 array([[99.26637976]]),
 array([[98.41905572]]),
 array([[97.61064365]]),
 array([[96.8393381]]),
 array([[96.10341784]]),
 array([[95.4012417]]),
 array([[94.73124474]]),
 array([[94.09193468]]),
 array([[93.48188845]]),
 array([[92.89974903]]),
 array([[92.34422234]]),
 array([[91.81407431]]),
 array([[91.30812814]]),
 array([[90.8252616]]),
 array([[90.36440453]]),
 array

In [ ]:
mod = LinearRegression()